In [1]:
!pip install python-terrier
!pip install transformers

import torch
import pyterrier as pt
import requests
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from pathlib import Path

In [2]:
pt.init()

Java started and loaded: pyterrier.java, pyterrier.terrier.java [version=5.11 (build: craig.macdonald 2025-01-13 21:29), helper_version=0.0.8]
C:\Users\basma\AppData\Local\Temp\ipykernel_21892\3651451178.py:1: DeprecationWarning: Call to deprecated method pt.init(). Deprecated since version 0.11.0.
java is now started automatically with default settings. To force initialisation early, run:
pt.java.init() # optional, forces java initialisation
  pt.init()


In [3]:
from pathlib import Path

# Load BM25 Index using PyTerrier
dataset = pt.datasets.get_dataset("irds:antique/test/non-offensive")
index = pt.index.IterDictIndexer(
    str(Path.cwd()),
    meta={
        "docno": 32,
        "text": 131072,
    },
    type=pt.index.IndexingType.MEMORY,
).index(dataset.get_corpus_iter())
bm25 = pt.BatchRetrieve(index, wmodel="BM25")

antique/test/non-offensive documents: 100%|██████████| 403666/403666 [01:25<00:00, 4695.34it/s]
C:\Users\basma\AppData\Local\Temp\ipykernel_21892\295160857.py:13: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  bm25 = pt.BatchRetrieve(index, wmodel="BM25")


In [4]:
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large")

C:\Users\basma\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
C:\Users\basma\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:

# def generate_reformulated_query(query, instructions):
#     reformulated_queries = []
#     for instruction in instructions:
#         prompt = f"{instruction}: {query}"
#         inputs = tokenizer(prompt, return_tensors="pt")
#         outputs = model.generate(**inputs, max_length=64, num_return_sequences=1)
#         reformulated_query = tokenizer.decode(outputs[0], skip_special_tokens=True)
#         reformulated_queries.append(reformulated_query)
#     return reformulated_queries

def generate_reformulated_query(query, instructions):
    reformulated_queries = []
    for instruction in instructions:
        prompt = f"{instruction}: {query}"
        inputs = tokenizer(prompt, return_tensors="pt")
        outputs = model.generate(**inputs, max_length=64, num_return_sequences=1)
        reformulated_query = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # *** Add this line to remove any potentially problematic colons: ***
        reformulated_query = reformulated_query.replace("?", "").replace('"', '').replace("'", "")
        print(reformulated_query)

        reformulated_queries.append(reformulated_query)
    return reformulated_queries

In [6]:
# Step 2: Retrieve Documents with BM25
def retrieve_documents(query, k=10):
    """Retrieve top-K documents using BM25 in PyTerrier."""
    results = bm25.search(query, k)
    return results[["docno", "score"]].values.tolist()

In [7]:
# Step 4: Ensemble Query Reformulations and Retrieve Documents
def ensemble_retrieve(query, instructions, k=10):
    """Combine multiple reformulated queries for retrieval."""
    reformulated_queries = generate_reformulated_query(query, instructions)
    print("Generated Queries:", reformulated_queries)
    all_results = []
    for reformulated_query in reformulated_queries:
        results = retrieve_documents(reformulated_query, k)
        # print("Retrieved Documents:", results)
        all_results.extend(results)

    aggregated_results = {}
    for docid, score in all_results:
        aggregated_results[docid] = aggregated_results.get(docid, 0) + score

    sorted_results = sorted(aggregated_results.items(), key=lambda x: x[1], reverse=True)
    return sorted_results[:k]

In [8]:
# Step 5: Ablation Study
def ablation_study(query, instructions, k=10):
    """Perform ablation by removing one instruction at a time and measuring impact."""
    full_results = ensemble_retrieve(query, instructions, k)
    full_score = sum(score for _, score in full_results)

    ablation_results = {}
    for i, removed_instruction in enumerate(instructions):
        reduced_instructions = [instr for j, instr in enumerate(instructions) if j != i]
        reduced_results = ensemble_retrieve(query, reduced_instructions, k)
        reduced_score = sum(score for _, score in reduced_results)

        ablation_results[removed_instruction] = full_score - reduced_score

    return ablation_results

def single_instruction_test(query, instructions, k=10):
    """Test each instruction separately and measure its individual impact."""
    instruction_scores = {}

    for instruction in instructions:
        print(f"Testing instruction: {instruction}")
        results = ensemble_retrieve(query, [instruction], k)
        total_score = sum(score for _, score in results)
        instruction_scores[instruction] = total_score

    return instruction_scores


In [14]:
import itertools

# Example Usage
query = "i want to know how does quantum mechanics work please help i need it please thanks"

# Manually input your paraphrased instructions here
paraphrases = [
    "Improve the search effectiveness by suggesting expansion terms for the query",
    "Recommend expansion terms for the query to improve search results",
    "Improve the search effectiveness by suggesting useful expansion terms for the query",
    "Maximize search utility by suggesting relevant expansion phrases for the query",
    "Enhance search efficiency by proposing valuable terms to expand the query",
    "Elevate search performance by recommending relevant expansion phrases for the query",
    "Boost the search accuracy by providing helpful expansion terms to enrich the query",
    "Increase the search efficacy by offering beneficial expansion keywords for the query",
    "Optimize search results by suggesting meaningful expansion terms to enhance the query",
    "Enhance search outcomes by recommending beneficial expansion terms to supplement the query"
]

allParaphraseSubsets = [subset for L in range(1, len(paraphrases) + 1) for subset in itertools.combinations(paraphrases, L)]

# print(allParaphraseSubsets)

[('Improve the search effectiveness by suggesting expansion terms for the query',), ('Recommend expansion terms for the query to improve search results',), ('Improve the search effectiveness by suggesting useful expansion terms for the query',), ('Maximize search utility by suggesting relevant expansion phrases for the query',), ('Enhance search efficiency by proposing valuable terms to expand the query',), ('Elevate search performance by recommending relevant expansion phrases for the query',), ('Boost the search accuracy by providing helpful expansion terms to enrich the query',), ('Increase the search efficacy by offering beneficial expansion keywords for the query',), ('Optimize search results by suggesting meaningful expansion terms to enhance the query',), ('Enhance search outcomes by recommending beneficial expansion terms to supplement the query',), ('Improve the search effectiveness by suggesting expansion terms for the query', 'Recommend expansion terms for the query to impro

In [10]:
# Run the ablation study
# ablation_results = ablation_study(query, paraphrases, k=10)
test = single_instruction_test(query, paraphrases, k=10)
# print("Ablation Study Results:", ablation_results)
# print("Single Instruction Test Results:", test)
for score in test:
    print(score, test[score])

Testing instruction: Improve the search effectiveness by suggesting expansion terms for the query
how does quantum mechanics work
Generated Queries: ['how does quantum mechanics work']
15:01:34.939 [main] WARN org.terrier.querying.ApplyTermPipeline -- The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.
Testing instruction: Recommend expansion terms for the query to improve search results
i want to know how does quantum mechanics work please help i need it please thanks
Generated Queries: ['i want to know how does quantum mechanics work please help i need it please thanks']
Testing instruction: Improve the search effectiveness by suggesting useful expansion terms for the query
how does quantum mechanics work
Generated Queries: ['how does quantum mechanics work']
Testing instruction: Maximize search utility by suggesting relev